<a href="https://colab.research.google.com/github/Nivratti/HeadPoseEstimation-WHENet/blob/master/HeadPoseEstimation_WHENet_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <font color="orange" size="+3">1. setup : </font>

In [1]:
!git clone https://github.com/Nivratti/HeadPoseEstimation-WHENet.git

Cloning into 'HeadPoseEstimation-WHENet'...
remote: Enumerating objects: 304, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 304 (delta 87), reused 278 (delta 71), pack-reused 0
Receiving objects: 100% (304/304), 25.89 MiB | 11.47 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [2]:
## Install retinaface -- face detection library 
# --no-dependencies : Ignore package dependencies -- otherwise it will install some specific version libraries -- it will take time and sometimes we need to restart kernel to laod new packages
!pip install --no-dependencies retinaface 
!pip install --no-dependencies utilpack ## it installing new pillow version -- and we need to restart -- so we are not installing any dependancies
!pip install slack_sdk
!pip install pdfrw
!pip install pycryptodome
!pip install PyMySQL

     |████████████████████████████████| 5.9MB 3.7MB/s 
     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 235kB 4.2MB/s 
ERROR: utilpack 2.0.0 requires gunicorn==20.1.0, which is not installed.
ERROR: utilpack 2.0.0 requires pdfrw==0.4, which is not installed.
ERROR: utilpack 2.0.0 requires pycryptodome==3.10.1, which is not installed.
ERROR: utilpack 2.0.0 requires PyMySQL==1.0.2, which is not installed.
ERROR: utilpack 2.0.0 has requirement Flask==2.0.1, but you'll have flask 1.1.4 which is incompatible.
ERROR: utilpack 2.0.0 has requirement matplotlib==3.3.4, but you'll have matplotlib 3.2.2 which is incompatible.
ERROR: utilpack 2.0.0 has requirement opencv-python==4.5.2.54, but you'll have opencv-python 4.1.2.30 which is incompatible.
ERROR: utilpack 2.0.0 has requirement Pillow==8.3.1, but you'll have pillow 7.1.2 which is incompatible.
ERROR: utilpack 2.0.0 has requirement requests==2.25.1, but you'll have requests 2.23.0 which is incom

In [3]:
## Auto time plugin
!pip install ipython-autotime

%load_ext autotime

time: 128 µs (started: 2021-07-18 11:54:57 +00:00)


In [4]:
!pip install nb_utils

time: 2.83 s (started: 2021-07-18 11:54:57 +00:00)


In [5]:
!pip install loguru

     |████████████████████████████████| 61kB 387kB/s 
time: 2.73 s (started: 2021-07-18 11:55:00 +00:00)


In [6]:
!pip install efficientnet

     |████████████████████████████████| 51kB 4.7MB/s 
time: 2.73 s (started: 2021-07-18 11:55:03 +00:00)


## <font color="orange" size="+3">2. Logger configuration: </font>


In [7]:
## Loge level, formatter
import sys
from loguru import logger

logger.remove() # remove previously added handlar - -to make a fresh start.

# enqueue=True : Asynchronous, Thread-safe, Multiprocess-safe
# backtrace=True, diagnose=True : Fully descriptive exceptions - Loguru helps you identify problems by allowing the entire stack trace to be displayed, including values of variables
logger.add(sys.stdout, colorize=True, format="<green>{time:HH:mm:ss}</green> | {level} | {name} | <level>{message}</level>", level="INFO", enqueue=True, backtrace=True, diagnose=True)

1

time: 26.8 ms (started: 2021-07-18 11:55:05 +00:00)


In [8]:
logger.info("Hi")

time: 936 µs (started: 2021-07-18 11:55:05 +00:00)
11:55:05 | INFO | __main__ | Hi


## <font color="orange" size="+3">3. Write Retinaface wrapper and init detector : </font>

Custom wrapper for reducing image size for speed improvement

In [9]:
#@title Define FaceDetection_RetinaFaceTF2
import cv2 
import imutils
from retinaface import RetinaFace

class FaceDetection_RetinaFaceTF2:
    def __init__(self, quality="normal"):
        self.face_detector = RetinaFace(quality=quality)

    def read(self, image_path):
        return self.face_detector.read(image_path)

    def detect_highlight_face(self, rgb_image, max_side_len=640, max_faces=1, return_boxformat="xywh"):
        ## Resize image
        scale_factor = None

        height, width, c = rgb_image.shape
        if width > max_side_len or height > max_side_len:
            if height > width:
                scale_factor = max_side_len / height

                resized = imutils.resize(rgb_image, height=max_side_len)
            else:
                scale_factor = max_side_len / width
                resized = imutils.resize(rgb_image, width=max_side_len)

            logger.info(f"scale_factor: {scale_factor}")
            # tn_image = resize((int(width * scale_factor), int(height * scale_factor)))

        if scale_factor:
            faces = self.face_detector.predict(resized)

            ## Rescaled face boxes -- restore
            if faces:
              for face in faces:
                  face["x1"] = int(face["x1"] / scale_factor)
                  face["y1"] = int(face["y1"] / scale_factor)
                  face["x2"] = int(face["x2"] / scale_factor)
                  face["y2"] = int(face["y2"] / scale_factor)
        else:
            faces = self.face_detector.predict(rgb_image)
    
        # print(faces)

        # faces is list of face dictionary
        # each face dictionary contains x1 y1 x2 y2 left_eye right_eye nose left_lip right_lip
        # faces=[{"x1":20,"y1":32, ... }, ...]

        if faces:
            result_img = self.face_detector.draw(rgb_image, faces)
        else:
            result_img = None
        # # save ([...,::-1] : rgb -> bgr )
        # cv2.imwrite("result_img.jpg",result_img[...,::-1])

        return (faces, result_img)

time: 3.08 s (started: 2021-07-18 11:55:05 +00:00)


In [10]:
face_detector = FaceDetection_RetinaFaceTF2()
logger.info("Success! Initialized face detector..")

model[normal quality] init ..
model success !
time: 19.8 s (started: 2021-07-18 11:55:09 +00:00)
11:55:28 | INFO | __main__ | Success! Initialized face detector..


## <h1><font color="orange" size="+3">4. Test detector : </font></h1>

In [12]:
## RetinaFaceTF2 -- takes less than 100 ms on GPU

# same with cv2.imread,cv2.cvtColor 
# rgb_image = face_detector.read("/content/HeadPoseEstimation-WHENet/Sample/maruti.jpg")
rgb_image = face_detector.read("/content/people_drinking.jpg")

faces, result_img = face_detector.detect_highlight_face(rgb_image, max_side_len=640)

logger.info(f"Total faces: {len(faces)}")
logger.info(f"faces: {faces}")

## RGB to BGR
cv2.imwrite("result_img.jpg", result_img[...,::-1])

11:58:04 | INFO | __main__ | scale_factor: 0.24691358024691357
11:58:04 | INFO | __main__ | Total faces: 11
11:58:04 | INFO | __main__ | faces: [{'x1': 1129, 'y1': 575, 'x2': 1255, 'y2': 716, 'left_eye': (292, 157), 'right_eye': (305, 158), 'nose': (300, 166), 'left_lip': (292, 169), 'right_lip': (302, 170)}, {'x1': 1782, 'y1': 700, 'x2': 1919, 'y2': 935, 'left_eye': (445, 196), 'right_eye': (448, 196), 'nose': (439, 208), 'left_lip': (448, 217), 'right_lip': (451, 216)}, {'x1': 238, 'y1': 368, 'x2': 328, 'y2': 477, 'left_eye': (65, 102), 'right_eye': (74, 101), 'nose': (70, 107), 'left_lip': (68, 112), 'right_lip': (75, 111)}, {'x1': 1632, 'y1': 587, 'x2': 1749, 'y2': 729, 'left_eye': (407, 157), 'right_eye': (416, 158), 'nose': (406, 164), 'left_lip': (407, 171), 'right_lip': (413, 172)}, {'x1': 453, 'y1': 558, 'x2': 562, 'y2': 712, 'left_eye': (115, 151), 'right_eye': (122, 152), 'nose': (113, 159), 'left_lip': (115, 166), 'right_lip': (120, 167)}, {'x1': 619, 'y1': 368, 'x2': 692, 

True

time: 618 ms (started: 2021-07-18 11:58:04 +00:00)


In [13]:
## Modified facebox format
face = [faces[0]["x1"], faces[0]["y1"], faces[0]["x2"], faces[0]["y2"]]
logger.info(f"face: {face} \n")

11:58:47 | INFO | __main__ | face: [1129, 575, 1255, 716] 

time: 2.1 ms (started: 2021-07-18 11:58:47 +00:00)


## <font color="orange" size="+3">2. Logger configuration: </font>


In [14]:
%cd "/content/HeadPoseEstimation-WHENet"

/content/HeadPoseEstimation-WHENet
time: 5.27 ms (started: 2021-07-18 11:58:53 +00:00)


In [15]:
import os
import cv2
import numpy as np
from PIL import Image
from utils import draw_axis


def process_detection( model, img_bgr, bbox, is_draw_axis=True, display_all_euler_angle="full"):

    img = img_bgr
    # y_min, x_min, y_max, x_max = bbox # yolo v3 face detector
    x_min, y_min, x_max, y_max = bbox # retinaface tf2

    # enlarge the bbox to include more background margin
    y_min = max(0, y_min - abs(y_min - y_max) / 10)
    y_max = min(img.shape[0], y_max + abs(y_min - y_max) / 10)
    x_min = max(0, x_min - abs(x_min - x_max) / 5)
    x_max = min(img.shape[1], x_max + abs(x_min - x_max) / 5)
    x_max = min(x_max, img.shape[1])

    img_cropped = img[int(y_min):int(y_max), int(x_min):int(x_max)]
    # cv2.imwrite("img_cropped.jpg", img_cropped)
    
    img_rgb = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2RGB)
    img_rgb = cv2.resize(img_rgb, (224, 224))
    img_rgb = np.expand_dims(img_rgb, axis=0)

    cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0,0,0), 2)
    yaw, pitch, roll = model.get_angle(img_rgb)
    yaw, pitch, roll = np.squeeze([yaw, pitch, roll])

    if is_draw_axis:
        draw_axis(img, yaw, pitch, roll, tdx=(x_min+x_max)/2, tdy=(y_min+y_max)/2, size = abs(x_max-x_min)//2 )

    if display_all_euler_angle == 'full':
        cv2.putText(img, "yaw: {}".format(np.round(yaw)), (int(x_min), int(y_min)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 255, 0), 1)
        cv2.putText(img, "pitch: {}".format(np.round(pitch)), (int(x_min), int(y_min) - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 255, 0), 1)
        cv2.putText(img, "roll: {}".format(np.round(roll)), (int(x_min), int(y_min)-30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 255, 0), 1)
    return (img, yaw, pitch, roll)


time: 120 ms (started: 2021-07-18 11:58:55 +00:00)


In [16]:
from whenet import WHENet

time: 624 ms (started: 2021-07-18 11:58:59 +00:00)


In [17]:
import os
import argparse

args = argparse.Namespace(
    snapshot="WHENet.h5",  # whenet snapshot path
    gpu='0',
)

## Set env
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

time: 6.4 ms (started: 2021-07-18 11:59:01 +00:00)


In [18]:
## Init head pose estimation model
whenet = WHENet(snapshot=args.snapshot)

16809984/16804768 [==============================] - 1s 0us/step
time: 3.02 s (started: 2021-07-18 11:59:04 +00:00)


In [22]:
%%time
# image_bgr = cv2.imread("/content/HeadPoseEstimation-WHENet/Sample/maruti.jpg")
image_bgr = cv2.imread("/content/people_drinking.jpg")
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

faces, result_img = face_detector.detect_highlight_face(image_rgb, max_side_len=640)

logger.info(f"Total faces: {len(faces)}")
# logger.info(f"faces: {faces}")

bboxes = []
for idx, face in enumerate(faces):
    bbox = [faces[idx]["x1"], faces[idx]["y1"], faces[idx]["x2"], faces[idx]["y2"]]
    bboxes.append(bbox)

## Visualize
img_pil = Image.fromarray(image_rgb)

for bbox in bboxes:
    out_img, yaw, pitch, roll = process_detection(whenet, image_bgr, bbox, args)

logger.info(f"Storing result in out.jpg ... \n")
cv2.imwrite("out.jpg", out_img)

12:01:56 | INFO | __main__ | scale_factor: 0.24691358024691357
12:01:56 | INFO | __main__ | Total faces: 11
12:01:57 | INFO | __main__ | Storing result in out.jpg ... 

CPU times: user 687 ms, sys: 16.4 ms, total: 704 ms
Wall time: 707 ms
time: 720 ms (started: 2021-07-18 12:01:56 +00:00)
